# Forcasting  

In this notebook we will predict the closing prices for the next month

In [1]:
# Import Libraries
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [2]:
# Load DataSet
data= pd.read_csv('./data/input_weekly_stock_market.csv')

# Reorder the columns
data = data[['companyName', 'date', 'openingPrice', 'highestPrice', 'lowestPrice', 'volume', 'closingPrice']]

data.head()

,companyName,date,openingPrice,highestPrice,lowestPrice,volume,closingPrice
0,AMEN BANK,2014-06-16,-0.104873,-0.105238,-0.105602,-0.111736,-0.105532
1,AMEN BANK,2014-06-23,-0.105491,-0.105851,-0.106085,-0.105668,-0.106023
2,AMEN BANK,2014-06-30,-0.105983,-0.106064,-0.106466,-0.099024,-0.106136
3,AMEN BANK,2014-07-07,-0.105983,-0.105814,-0.106708,-0.111046,-0.106036
4,AMEN BANK,2014-07-14,-0.105870,-0.105826,-0.105818,-0.110070,-0.106023


In [3]:
print(data.shape)

(38629, 7)


In [4]:
# Extracting the closingPrice column for sequence creation
closing_prices = data['closingPrice'].values

In [10]:
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    data = np.array(data)

    for i in range(len(data)-seq_length):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        sequences.append(x)
        labels.append(y)

    return np.array(sequences), np.array(labels)

seq_length = 5
X, y = create_sequences(data, seq_length)

print("Sequences")
print(X)
print("Labels")
print(y)

Sequences
[[['AMEN BANK' '2014-06-16' -0.1048733843494207 ... -0.1056021828456061
   -0.1117356631601544 -0.1055317678390078]
  ['AMEN BANK' '2014-06-23' -0.105491423567736 ... -0.1060850703275353
   -0.1056680500569942 -0.1060230512054017]
  ['AMEN BANK' '2014-06-30' -0.1059833323333338 ... -0.1064662972869531
   -0.0990235176700086 -0.1061364242899541]
  ['AMEN BANK' '2014-07-07' -0.1059833323333338 ... -0.1067077410279177
   -0.1110455911991147 -0.1060356482147964]
  ['AMEN BANK' '2014-07-14' -0.1058698149258882 ... -0.1058182114559428
   -0.1100698485706007 -0.1060230512054017]]

 [['AMEN BANK' '2014-06-23' -0.105491423567736 ... -0.1060850703275353
   -0.1056680500569942 -0.1060230512054017]
  ['AMEN BANK' '2014-06-30' -0.1059833323333338 ... -0.1064662972869531
   -0.0990235176700086 -0.1061364242899541]
  ['AMEN BANK' '2014-07-07' -0.1059833323333338 ... -0.1067077410279177
   -0.1110455911991147 -0.1060356482147964]
  ['AMEN BANK' '2014-07-14' -0.1058698149258882 ... -0.1058182